# Imports

In [ ]:
import pandas as pd

# Data Collection

In [ ]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_X_poc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_Y_poc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/test_X_poc.csv

--2021-09-16 16:39:40--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_X_poc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.64.15
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.64.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312331 (305K) [text/csv]
Saving to: ‘train_X_poc.csv’

train_X_poc.csv     100%[===================>] 305.01K   562KB/s    in 0.5s    

2021-09-16 16:39:41 (562 KB/s) - ‘train_X_poc.csv’ saved [312331/312331]

--2021-09-16 16:39:41--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_Y_poc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (n

In [ ]:
train_X_df = pd.read_csv('train_X_poc.csv')
train_Y_df = pd.read_csv('train_Y_poc.csv', header=None).squeeze()
test_X_df = pd.read_csv('test_X_poc.csv')

# Understanding Data

In [ ]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   peppermint                1125 non-null   float64
 1   dittany                   1125 non-null   float64
 2   lavender                  1125 non-null   float64
 3   ginger_root_width(mm)     1125 non-null   float64
 4   powdered_dragon_claw(mg)  1125 non-null   float64
 5   fluxweed(mg)              1125 non-null   float64
 6   drops_of_tree_wood_oil    1125 non-null   float64
 7   fire_seed                 1125 non-null   float64
 8   rose_petal                1125 non-null   float64
 9   daisy_root_width          1125 non-null   float64
 10  daisy_root_height         1125 non-null   float64
 11  water(ml)                 1125 non-null   float64
 12  dragon_blood(ml)          1125 non-null   float64
 13  porcupine_quills          1125 non-null   float64
 14  drops_of

In [ ]:
train_X_df.head()

,peppermint,dittany,lavender,ginger_root_width(mm),powdered_dragon_claw(mg),fluxweed(mg),drops_of_tree_wood_oil,fire_seed,rose_petal,daisy_root_width,daisy_root_height,water(ml),dragon_blood(ml),porcupine_quills,drops_of_red_wine,gillyweed(mg),honey(mg),morning_dew,ginger_root_height(mm),leech_juice(ml)
0,1.0,0.0,1.0,0.172230,0.10,0.857143,0.426243,0.0,0.0,0.722222,0.714286,0.656417,0.111111,0.000000,0.055556,56.0,4.1,1.0,0.272447,0.800000
1,1.0,1.0,1.0,0.338451,0.45,0.142857,0.200160,1.0,0.0,0.111111,0.500000,0.233957,1.000000,0.666667,0.388889,56.0,4.3,0.0,0.404310,0.175000
2,1.0,1.0,1.0,0.638184,0.60,1.000000,0.743453,0.0,1.0,0.388889,0.714286,0.635027,0.555556,0.888889,0.277778,59.0,2.5,0.0,0.416624,0.841667
3,1.0,0.0,0.0,0.830441,0.50,0.714286,0.194816,1.0,0.0,0.611111,0.500000,0.649064,0.444444,0.277778,0.055556,38.0,4.8,1.0,0.744997,0.416667
4,1.0,1.0,0.0,0.080774,0.80,0.714286,0.728220,1.0,1.0,0.333333,0.714286,0.307487,0.222222,0.277778,0.555556,53.0,3.5,1.0,0.284248,0.600000


# Implementation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV

pipe = Pipeline(steps=[('scaler', RobustScaler()),
                       ('selector', SelectKBest(score_func=f_regression)),
                       ('classifier', LogisticRegression(solver='sag', max_iter=2000))])

In [ ]:
param_distributions = [
              {
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
                'selector__k': [10, 15, 18],
                'classifier': [LogisticRegression(solver='lbfgs', max_iter=5000, multi_class='ovr')],
                'classifier__penalty': ['l2', 'none']
              },
              {
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
                'classifier': [KNeighborsClassifier()],
                'classifier__n_neighbors': range(1,20),
                'classifier__p': [1, 2, 3, 4]
              }
            ]

grid_search_cv = GridSearchCV(pipe, param_grid=param_distributions, scoring='f1_weighted', refit=True, cv=5) 
grid_search_cv.fit(train_X_df, train_Y_df)
print(grid_search_cv.best_params_)

{'classifier': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='ovr', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'classifier__penalty': 'none', 'scaler': RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True), 'selector__k': 10}


In [ ]:
grid_search_cv.score(train_X_df, train_Y_df)

0.8954307065789857

In [ ]:
predicted_test_Y = grid_search_cv.predict(test_X_df)

Writing the predicted values to CSV file

In [ ]:
pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_poc.csv', header=None, index=False)